# LangGraph ReAct Agent mit Azure OpenAI

Dieses Notebook zeigt, wie man einen einfachen ReAct Agent mit LangGraph und Azure OpenAI baut, der streamt.

In [1]:
# Installation (falls nötig)
# !pip install langgraph langchain langchain-openai langchain-community

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent
from langgraph.graph import START, END
from langchain.tools import tool
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import asyncio

In [3]:
load_dotenv()

True

## 1. Azure OpenAI Setup

In [4]:
# Azure OpenAI Konfiguration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-api-key")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-resource.openai.azure.com/")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

# LLM initialisieren
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    temperature=0.7,
    streaming=True,  # Für Streaming-Unterstützung
    callbacks=[StreamingStdOutCallbackHandler()]  # Live-Output
)

## 2. Einfache Tools definieren

In [5]:
@tool
def calculator(expression: str) -> str:
    """Führt einfache mathematische Berechnungen aus.
    
    Args:
        expression: Eine mathematische Expression wie "2 + 3 * 4"
        
    Returns:
        Das Ergebnis der Berechnung als String
    """
    try:
        # Sicherheitscheck - nur erlaubte Operationen
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Fehler: Nur Zahlen und Grundrechenarten (+, -, *, /) erlaubt"
        
        result = eval(expression)
        return f"Ergebnis: {result}"
    except Exception as e:
        return f"Fehler bei der Berechnung: {str(e)}"

@tool
def search_web(query: str) -> str:
    """Simuliert eine Web-Suche (für Demo-Zwecke).
    
    Args:
        query: Suchbegriff
        
    Returns:
        Simuliertes Suchergebnis
    """
    # In der Realität würde hier eine echte Such-API verwendet werden
    mock_results = {
        "wetter berlin": "Das Wetter in Berlin ist sonnig, 22°C",
        "python tutorial": "Python ist eine Programmiersprache. Besuchen Sie python.org für Tutorials",
        "langchain": "LangChain ist ein Framework für LLM-Anwendungen",
        "langgraph": "LangGraph ist ein Framework für agentische Workflows mit Graph-Struktur"
    }
    
    for key, result in mock_results.items():
        if key in query.lower():
            return f"Suchergebnis für '{query}': {result}"
    
    return f"Keine Ergebnisse gefunden für: {query}"

# Tools-Liste
tools = [calculator, search_web]

### ReAct Agent erstellen

Im folgenden Abschnitt erstellen wir einen ReAct-Agenten mit LangChain/ LangGraph. Verwende `create_agent` statt des älteren `create_react_agent`-Helpers.

In [6]:
# ReAct Agent mit LangGraph erstellen
# Dies erstellt automatisch einen vollständigen Graph mit:
# - State Management
# - Tool-Calling
# - ReAct Logik
# - Streaming-Unterstützung

app = create_agent(llm, tools)

# Graph visualisieren (optional)
print("Graph Structure:")
print(app.get_graph().draw_ascii())

Graph Structure:
        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +-------+           
          | model |           
          +-------+.          
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


## 4. Agent testen (ohne Streaming)

In [7]:
# Einfacher Test ohne Streaming
result = app.invoke({
    "messages": [{"role": "user", "content": "Was ist 15 + 27 mal 3?"}]
})

print("Finale Antwort:")
print(result["messages"][-1].content)

Das Ergebnis von 15 + 27 mal 3 ist 96.Finale Antwort:
Das Ergebnis von 15 + 27 mal 3 ist 96.


## 5. Agent mit Streaming

LangGraph bietet integrierte Streaming-Unterstützung über `astream()` und `astream_events()`.

In [ ]:
# Streaming-Funktion
async def run_agent_streaming(query: str):
    """Führt den Agent mit Streaming aus"""
    print(f"Frage: {query}")
    print("=" * 50)
    
    async for event in app.astream_events(
        {"messages": [{"role": "user", "content": query}]},
        version="v1"
    ):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="", flush=True)
        elif kind == "on_tool_start":
            print(f"\n🔧 Using tool: {event['name']}")
        elif kind == "on_tool_end":
            print(f"✅ Tool result: {event['data']['output'][:100]}...")

# Beispiel mit Streaming
await run_agent_streaming("Berechne 42 * 7 und suche nach 'langgraph'")

Frage: Berechne 42 * 7 und suche nach 'langgraph'

🔧 Using tool: search_web

🔧 Using tool: calculator


TypeError: 'ToolMessage' object is not subscriptable

Das Ergebnis der Berechnung von 42 * 7 ist 294. 

Zur Suche nach "langgraph": LangGraph ist ein Framework für agentische Workflows mit Graph-Struktur. Wenn du mehr Details möchtest, kann ich gerne weiter helfen.

## 6. Detailliertes Streaming mit Events

In [ ]:
async def run_agent_detailed_streaming(query: str):
    """Zeigt alle Streaming-Events detailliert"""
    print(f"Frage: {query}")
    print("=" * 60)
    
    async for event in app.astream_events(
        {"messages": [{"role": "user", "content": query}]},
        version="v1"
    ):
        kind = event["event"]
        print(f"Event: {kind}")
        
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(f"  Content: {repr(content)}")
        elif kind == "on_tool_start":
            print(f"  Tool: {event['name']}")
            print(f"  Input: {event['data']['input']}")
        elif kind == "on_tool_end":
            print(f"  Tool Result: {event['data']['output'][:50]}...")
        elif kind == "on_chain_start":
            print(f"  Chain: {event['name']}")
        elif kind == "on_chain_end":
            print(f"  Chain End: {event['name']}")
        
        print("-" * 40)

# Detailliertes Beispiel
await run_agent_detailed_streaming("Was ist 10 + 5?")

Frage: Was ist 10 + 5?
Event: on_chain_start
  Chain: LangGraph
----------------------------------------
Event: on_chain_start
  Chain: model
----------------------------------------
Event: on_chat_model_start
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
----------------------------------------
Event: on_chat_model_stream
-------------------------------

TypeError: 'ToolMessage' object is not subscriptable

10 + 5 ergibt 15.

## 7. State-Streaming (nur finale Updates)

In [10]:
async def run_agent_state_streaming(query: str):
    """Streamt nur die finalen State-Updates"""
    print(f"Frage: {query}")
    print("=" * 50)
    
    async for state in app.astream(
        {"messages": [{"role": "user", "content": query}]}
    ):
        # Zeigt nur die neueste Nachricht
        if state["messages"]:
            latest_msg = state["messages"][-1]
            if hasattr(latest_msg, 'content') and latest_msg.content:
                print(f"State Update: {latest_msg.content[:100]}...")
        print("-" * 30)

# State-Streaming Beispiel
await run_agent_state_streaming("Erkläre kurz was LangGraph ist")

Frage: Erkläre kurz was LangGraph ist


KeyError: 'messages'

## 8. Einfacher Chat-Loop

In [ ]:
async def chat_loop():
    """Einfacher Chat-Loop für Interaktion"""
    print("LangGraph ReAct Agent Chat (type 'quit' to exit)")
    print("-" * 50)
    
    while True:
        user_input = input("Du: ")
        if user_input.lower() in ['quit', 'exit', 'q']:
            break
            
        await run_agent_streaming(user_input)
        print("\n" + "="*60 + "\n")

# Chat starten (kommentiert aus, da input() in Notebook problematisch ist)
# await chat_loop()

## 9. Custom Graph (für erweiterte Anwendungsfälle)

Wenn du mehr Kontrolle brauchst, kannst du auch einen eigenen Graph bauen:

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode

# Custom State (erweitert MessagesState)
class AgentState(MessagesState):
    pass

# Tool Node
tool_node = ToolNode(tools)

# Agent Node
def agent_node(state: AgentState):
    """Agent denkt und entscheidet über Tool-Nutzung"""
    messages = state["messages"]
    response = llm.bind_tools(tools).invoke(messages)
    return {"messages": [response]}

# Bedingung für Tool-Nutzung
def should_use_tools(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return END

# Custom Graph bauen
custom_graph = StateGraph(AgentState)
custom_graph.add_node("agent", agent_node)
custom_graph.add_node("tools", tool_node)
custom_graph.add_edge(START, "agent")
custom_graph.add_conditional_edges("agent", should_use_tools)
custom_graph.add_edge("tools", "agent")

custom_app = custom_graph.compile()

# Custom Graph testen
result = custom_app.invoke({
    "messages": [HumanMessage(content="Was ist 25 * 4?")]
})

print("Custom Graph Result:")
print(result["messages"][-1].content)

In früheren Versionen wurde `create_react_agent()` verwendet. Aktuell nutze `create_agent()` und übergebe das LLM sowie die Tools. Der Agent unterstützt Streaming und State-Graph-Visualisierung.